***Defect detection model which can only dectect defects***

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Define parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = 6
epochs = 20

In [3]:
# Image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1620 images belonging to 6 classes.
Found 180 images belonging to 6 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,399,174 (169.37 MB)

 Trainable params: 44,399,174 (169.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


C:\Users\mohit\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 170s 3s/step - accuracy: 0.1886 - loss: 2.4381 - val_accuracy: 0.3063 - val_loss: 1.6494
Epoch 2/20
 1/50 ━━━━━━━━━━━━━━━━━━━━ 1:39 2s/step - accuracy: 0.3438 - loss: 1.4166

C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


50/50 ━━━━━━━━━━━━━━━━━━━━ 27s 505ms/step - accuracy: 0.3438 - loss: 1.4166 - val_accuracy: 0.4500 - val_loss: 1.2045
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5725 - loss: 1.1095 - val_accuracy: 0.7250 - val_loss: 0.7704
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6875 - loss: 0.7620 - val_accuracy: 0.7000 - val_loss: 0.9544
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.7226 - loss: 0.7397 - val_accuracy: 0.7625 - val_loss: 0.6379
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 284ms/step - accuracy: 0.6875 - loss: 0.6028 - val_accuracy: 0.8000 - val_loss: 0.5244
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.8066 - loss: 0.5592 - val_accuracy: 0.5813 - val_loss: 1.5690
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9375 - loss: 0.1973 - val_accuracy: 0.5000 - val_loss: 3.7933
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.6128 - loss: 1.0666 - val_accuracy: 0.8813 - val_loss: 

In [ ]:
# Load the best model
best_model = load_model('best_model.keras')

# Define the target names
target_names = list(train_generator.class_indices.keys())

# Start video capture
cap = cv2.VideoCapture(0)  # Change '0' to the correct camera index if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (img_height, img_width))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)

    # Make prediction
    preds = best_model.predict(img)
    pred_class = np.argmax(preds, axis=1)[0]
    pred_label = target_names[pred_class]

    # Display the prediction on the frame
    cv2.putText(frame, f'Defect: {pred_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Real-time Defect Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━

***Defect detection model which can dectect defects and log the data into csv file***

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from datetime import datetime
import csv

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Define parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = 6
epochs = 20

In [7]:
# Image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1620 images belonging to 6 classes.
Found 180 images belonging to 6 classes.


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

C:\Users\mohit\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,399,174 (169.37 MB)

 Trainable params: 44,399,174 (169.37 MB)

...

Epoch 1/20


C:\Users\mohit\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.1776 - loss: 2.3708 - val_accuracy: 0.6875 - val_loss: 1.0028
Epoch 2/20
 1/50 ━━━━━━━━━━━━━━━━━━━━ 1:23 2s/step - accuracy: 0.5000 - loss: 1.0973

C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5000 - loss: 1.0973 - val_accuracy: 0.5500 - val_loss: 1.1986
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.6038 - loss: 0.9870 - val_accuracy: 0.5813 - val_loss: 1.1565
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5938 - loss: 0.9967 - val_accuracy: 0.6500 - val_loss: 1.0100
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.5784 - loss: 1.0381 - val_accuracy: 0.7937 - val_loss: 0.6342
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6250 - loss: 0.9442 - val_accuracy: 0.7000 - val_loss: 0.8375
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.7976 - loss: 0.5912 - val_accuracy: 0.8625 - val_loss: 0.4645
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8750 - loss: 0.4906 - val_accuracy: 0.6000 - val_loss: 0.9408
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.8317 - loss: 0.4506 - val_accuracy: 0.5437 - val_loss: 1.0720
E

In [7]:
# Save the model after training
model.save('defect_detection_model.keras')

In [2]:
# Load the trained model
from tensorflow.keras.models import load_model
best_model = load_model('defect_detection_model.keras')

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from datetime import datetime
import csv

# Load the trained model
best_model = load_model('defect_detection_model.keras')

# Define the target names
target_names = list(train_generator.class_indices.keys())

# Open CSV file for logging
log_file = open('defect_log.csv', mode='w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['Timestamp', 'Defect', 'Confidence'])

# Function to draw bounding box
def draw_bounding_box(frame, label, confidence, x, y, w, h):
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    text = f'{label}: {confidence:.2f}'
    cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Function to apply sliding window
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Parameters for sliding window
window_size = (img_width, img_height)
step_size = 64

# Start video capture
cap = cv2.VideoCapture(0)  # Change '0' to the correct camera index if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detected = False
    for (x, y, window) in sliding_window(frame, step_size, window_size):
        if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
            continue

        # Preprocess the window
        img = window.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        # Make prediction
        preds = best_model.predict(img)
        pred_class = np.argmax(preds, axis=1)[0]
        pred_label = target_names[pred_class]
        confidence = preds[0][pred_class]

        # Only consider predictions with high confidence
        if confidence > 0.5:  # Adjust the threshold as needed
            draw_bounding_box(frame, pred_label, confidence, x, y, window_size[0], window_size[1])
            detected = True

            # Log the defect with timestamp
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_writer.writerow([timestamp, pred_label, confidence])

    # Display the frame
    cv2.imshow('Real-time Defect Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
log_file.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

***Defect detection using yolov5***

In [15]:
!pip install opencv-python-headless torch torchvision pandas


Defaulting to user installation because normal site-packages is not writeable


In [16]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


Defaulting to user installation because normal site-packages is not writeable


In [20]:
!pip install virtualenv

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.7 MB 660.6 kB/s eta 0:00:09
    --------------------------------------- 0.1/5.7 MB 1.1 MB/s eta 0:00:06
   - -------------------------------------- 0.2/5.7 MB 1.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.3/5.7 MB 1.6 MB/s eta 0:00:04
   --- ------------------------------------ 0.6/5.7 MB 2.2 MB/s eta 0:00:03
   ----- ---------------------------------- 0.7/5.7 MB 2.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.0/5.7 MB 3.1 MB/s eta 0:00:02
   --------- ------------------------------ 1.3/5.7 MB 3.4 MB/s eta 0:00:02
   ---------- ----------------------------- 1.6/5.7 MB 3.7 MB/s eta 0:00:02
   ------------ --------------------------- 1.8/5.7 MB 3.8 MB/s eta 0:00:02
   -------------- ------------------------- 2.0/5.7 MB 3.9 MB/s eta 0:00:01
   ------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
!python -m virtualenv venv

created virtual environment CPython3.11.7.final.0-64 in 24525ms
  creator CPython3Windows(dest=C:\Users\mohit\Desktop\DQ-Learning based DNRP\defect detection\yolov5\yolov5\venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\mohit\AppData\Local\pypa\virtualenv)
    added seed packages: pip==24.1, setuptools==70.1.0, wheel==0.43.0
  activators BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [22]:
import sys
import os

# Define the paths for the virtual environment
venv_path = os.path.join(os.getcwd(), 'venv')
activate_this = os.path.join(venv_path, 'Scripts', 'activate_this.py')

# Activate the virtual environment
exec(open(activate_this).read(), {'__file__': activate_this})

# Add the virtual environment to the sys.path
sys.path.insert(0, os.path.join(venv_path, 'Lib', 'site-packages'))


In [23]:
!{venv_path}/Scripts/pip install -r yolov5/requirements.txt

'C:\Users\mohit\Desktop\DQ-Learning' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
# Step 1: Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Step 2: Create data.yaml file
data_yaml = """
train: ../dataset/train
val: ../dataset/test

nc: 6
names: ['Crazing', 'Inclusions', 'Patches', 'Pitted Surface', 'Rolled in Scale', 'Scratches']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

# Step 3: Train YOLOv5
!{venv_path}/Scripts/python train.py --img 416 --batch 16 --epochs 50 --data data.yaml --cfg yolov5s.yaml --weights yolov5s.pt --name defect_detection


C:\Users\mohit\Desktop\DQ-Learning based DNRP\defect detection\yolov5\yolov5\yolov5


Cloning into 'yolov5'...
'C:\Users\mohit\Desktop\DQ-Learning' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from datetime import datetime
import csv

# Load and preprocess the dataset (convert to YOLO format)

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Parameters
img_height, img_width = 416, 416  # YOLOv3 input size
batch_size = 16
epochs = 50

# Data generators for YOLO format
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the YOLOv3 model
yolo_model = tf.keras.applications.YOLOv3(input_shape=(img_height, img_width, 3), include_top=False, weights='yolov3.weights')

# Add custom layers on top of YOLOv3
x = yolo_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.models.Model(inputs=yolo_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_yolov3_model.keras', monitor='val_loss', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

# Save the model
model.save('defect_detection_yolov3_model.keras')


Found 1620 images belonging to 6 classes.
Found 180 images belonging to 6 classes.


AttributeError: module 'keras.api.applications' has no attribute 'YOLOv3'

***yolov5 human detaction***

In [26]:
import torch
import cv2
import numpy as np
from datetime import datetime
import csv

# Load the trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt')

# Open CSV file for logging
log_file = open('defect_log.csv', mode='w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['Timestamp', 'Defect', 'Confidence'])

# Function to draw bounding box
def draw_bounding_box(frame, label, confidence, x, y, w, h):
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    text = f'{label}: {confidence:.2f}'
    cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Start video capture
cap = cv2.VideoCapture(0)  # Change '0' to the correct camera index if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(img)

    # Process results
    for result in results.xyxy[0]:  # x1, y1, x2, y2, confidence, class
        x1, y1, x2, y2, conf, cls = result
        x, y, w, h = int(x1), int(y1), int(x2 - x1), int(y2 - y1)
        label = model.names[int(cls)]
        confidence = conf.item()

        if confidence > 0.7:  # Adjust threshold as needed
            draw_bounding_box(frame, label, confidence, x, y, w, h)

            # Log the defect with timestamp
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_writer.writerow([timestamp, label, confidence])

    # Display the frame
    cv2.imshow('Real-time Defect Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
log_file.close()


Using cache found in C:\Users\mohit/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-1 Python-3.11.7 torch-2.2.2+cpu CPU

100%|█████████████████████████████████████████████████████████████████████████████| 14.1M/14.1M [00:03<00:00, 4.48MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


KeyboardInterrupt: 